In [1]:
from autogen import ConversableAgent
from openai import OpenAI
from pathlib import Path

import LLM_configs
import Prompt_dics
from LLM_configs import *

In [2]:
# 加载 .env 文件
load_dotenv()

True

## 文件提取

In [3]:
# Kimi的文件接口
client = OpenAI(
    api_key = os.environ.get("Kimi_API_KEY"),
    base_url = "https://api.moonshot.cn/v1",
)

In [4]:
# 使用kimi api读取
def file_extraction(file_path):
    file_object = client.files.create(file=Path(file_path), purpose="file-extract")
    file_content = client.files.content(file_id=file_object.id).text
    return file_content

In [5]:
text = file_extraction("test files/象上-文稿-转写结果.docx")

In [6]:
PPT = file_extraction("test files/上海象上投资.pptx")

## Agent设计

In [7]:
quant_dict = {
    'quant_idea':[],
    'factor_composition':[],
    'factor_explanation':[],
    'model_explanation':[],
    'factor_update':[],
    'strategy_execution':[]
}

In [8]:
def report_writing(weak_config, strong_config,prompts,component_dict, requirements):
    extraction_assistant = ConversableAgent(
        name = "信息提取员",
        system_message = "你是一个信息提取员，任务是将我需要的信息，从我给你的一大段文字中提取出来，并返回给我",
        llm_config = weak_config
    )
    
    integrate_assistant = ConversableAgent(
        name="报告撰写员",
        system_message=f"你是一个报告撰写员，需要根据我发给你的字段，根据我的要求，详细的写出一份报告，并返回给我,以下是要求：不需要面面俱到，写出提供材料中包含的即可：{requirements}",
        llm_config = strong_config
    )
    
    for prompt, component in zip(prompts, list(component_dict.keys())):
        reply = extraction_assistant.generate_reply(messages=[{"content": text, "role":"system"},
                                                      {"content": prompt, "role": "user"}])
        component_dict[component].append(reply)
    
    final_report = integrate_assistant.generate_reply(messages=[{"content": str(quant_dict), "role": "system"},
                                                                {
                                                                    "content": "按需求生成报告，不要分段，写成一整段，要求逻辑清晰，细节完整，长度越长越好，但不要有过多的你自由发挥的解释，客观记录为主, 不要出现评价语言，例如“这体现了XXXX",
                                                                    "role": "user"}])
    
    cost_list = [extraction_assistant.get_total_usage(),integrate_assistant.get_total_usage()]
    
    return final_report, component_dict, cost_list
        
        
    

In [9]:
Ali_report, Quant_component_dic, cost_list = report_writing(LLM_configs.kimi_weak_llm_config, LLM_configs.Qwen_strong_llm_config, Prompt_dics.Strategy_prompt_dic['Simple Quant Strategy'],quant_dict, Prompt_dics.Strategy_requirements_dic['Simple Quant Strategy'])


>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...


In [10]:
print(Ali_report)

管理人的量化策略整体思路围绕着综合运用手工挖掘与机器学习技术，以实现因子的有效识别与策略优化。策略核心在于结合基本面因子（占比40%）、另类因子（占比10%）与量价因子（占比50%）的多元视角，通过约三五十个精心挑选的因子，构建出逻辑性强、适应市场变化的量化模型。这些因子的挖掘主要依赖于人工逻辑筛选，同时正逐步引入机器学习方法以增强因子发现的效率与深度，虽然具体算法未明，但这一结合方式显示了管理人在追求因子质量与创新上的努力。

在因子处理上，管理人首先通过单因子测试，确保因子具备显著的多空收益和良好的夏普比率，随后依据因子间相关性及历史表现调整权重，实现因子的有效合成与优化。模型构建方面，管理人采取了线性排序打分模型与机器学习模型的混合策略，前者用于日常的股票排序与评分，后者则专注于预测子策略表现以动态调整策略权重，同时探索舆情数据的应用潜力，尽管实际应用尚在开发阶段。这种双轨制模型设计旨在平衡模型的解释力与预测能力，提高策略的灵活性与适应性。

新因子的入库标准严谨，需经过模拟盘的长期观察，高频因子约两至三个月，低频因子则延长至半年，确保因子的稳健性。入库前还需满足严格的绩效指标，如多空收益与夏普比率，以此保证因子池的质量。此外，管理人对因子实行持续监控与动态调整机制，即使暂时失效的因子也会视市场环境变化而考虑重新启用。

执行层面，管理人通过细致的操作实现了策略的精准落地。策略涵盖约200至300只股票，保持与指数的中性定位，行业偏离度控制在5%至10%，风格偏离度维持在基准的15%左右，体现了策略的均衡与纪律性。年化双边换手率维持在60至80倍，显示了适度的交易活跃度，而持股数量与市值暴露的管理则反映了对风险的严格控制。调仓策略灵活，既保持每日微调，也适时进行大比例调整，确保策略的动态优化。

对于对冲，虽然具体对冲方法未明确提及，但基于量化中性策略的实施，可以推断管理人运用了相应的市场中性手段，如使用期货、期权或其他金融衍生品来对冲市场风险，确保策略表现独立于市场方向，减少敞口风险。

综上所述，管理人的量化策略展现了从因子挖掘、处理、模型构建到执行的全面且细致的流程，结合了传统逻辑与现代机器学习的优势，旨在通过高度优化的策略配置捕捉市场机会，同时严格控制风险，实现稳定的投资回报。


In [11]:
cost_list

[{'total_cost': 0.4515720000000001,
  'moonshot-v1-8k': {'cost': 0.4515720000000001,
   'prompt_tokens': 35944,
   'completion_tokens': 1687,
   'total_tokens': 37631}},
 {'total_cost': 0.16332000000000002,
  'qwen-max': {'cost': 0.16332000000000002,
   'prompt_tokens': 2289,
   'completion_tokens': 598,
   'total_tokens': 2887}}]

In [14]:
Quant_component_dic

{'quant_idea': ['在对话中，说话人1询问了关于量化策略的因子挖掘方式，说话人3回答说：“目前大部分都是手工挖掘出来的，然后有一位同事现在正在做机器学习的那部分的。” 这表明该公司在量化策略的因子挖掘上，主要是以手工挖掘为主，但也在探索使用机器学习方法。此外，说话人1还问到因子是否有逻辑性，说话人3确认说：“对，明白。我们目前的话，我们因子都是有逻辑的因子。” 这说明该公司在量化策略中对因子的可解释性有一定的要求。'],
 'factor_composition': ['管理人的策略中，因子组成包括基本面因子占比40%，另类因子占比10%，量价因子占比50%。至于因子的总数，文档中提到“具体的个数我们没去统计，可能三五十个因子应该是有的”。'],
 'factor_explanation': ['在提供的文本中，管理人对其不同因子类型的解释主要涉及以下几个方面：\n\n1. 因子的构成：管理人提到他们的量化模型主要基于三类因子，分别是基本面因子（占比40%）、另类因子（占比10%）和量价因子（占比50%）。\n\n2. 因子的挖掘：管理人明确表示，目前大部分因子是手工挖掘出来的，但有一位同事正在研究机器学习的部分。\n\n3. 因子的逻辑性：管理人强调，他们的因子都是有逻辑的，即每个因子都有其背后的逻辑支持。\n\n4. 因子的测试与筛选：管理人提到，他们会对因子进行单因子测试，只有通过一定检验的因子，如多空收益和SL等数值达到要求，才会被加入到数据库中。\n\n5. 因子的组合与权重调整：管理人解释说，他们会将因子与其他因子结合，考虑其相关性和对组合的贡献，基于历史表现和因子间的相关性来调整不同因子的权重。\n\n6. 因子的更新与淘汰：管理人表示，他们会持续跟踪因子的表现，并根据其与历史表现的一致性进行权重调整，如果因子表现不佳，可能会被暂时淘汰，但也有可能在不同市场风格下重新变得有效。\n\n7. 数据来源：管理人提到他们使用的量化数据主要来自通联，而不是同花顺，尽管同花顺的数据基于Level-1数据，但他们发现同花顺的另类数据对他们的策略增强效果一般。\n\n这些信息提供了对管理人如何基于不同数据得出其量化模型中不同因子的详细解释。'],
 'model_explanation': ['在提供的对话记录中，管理人讨论了他们的建模方法和使用的算法

In [36]:
cost_list

[{'total_cost': 0.37473600000000007,
  'moonshot-v1-8k': {'cost': 0.37473600000000007,
   'prompt_tokens': 29936,
   'completion_tokens': 1292,
   'total_tokens': 31228}},
 {'total_cost': 0.1404,
  'qwen-max': {'cost': 0.1404,
   'prompt_tokens': 1923,
   'completion_tokens': 529,
   'total_tokens': 2452}}]

In [16]:
extraction_assistant = ConversableAgent(
        name = "信息提取员",
        system_message = "你是一个信息提取员，任务是将我需要的信息，从我给你的一大段文字中提取出来，并返回给我",
        llm_config = LLM_configs.kimi_weak_llm_config
    )

In [18]:
extraction_assistant.generate_reply(messages=[{"content": text, "role":"system"},
                                                      {"content": "从文件中提取出管理人的建模方法，需要明确提示是线性还是机器学习模型或者是两者都有及其偏好。其中线性模型包括：排序打分和简单回归，详述他们的建模方法论, 并提取出他们使用了的算法和用途。不要把因子挖掘的部分写进去，只需要说建模", "role": "user"}])


>>>>>>>> USING AUTO REPLY...


'根据文件内容，管理人的建模方法包括线性模型和机器学习模型。以下是详细的建模方法论和使用的算法及其用途：\n\n1. **线性模型**：\n   - **排序打分**：管理人使用线性模型对股票进行排序打分。在单因子测试中，如果因子满足一定的检验标准（如多空收益、SL等数值），则将其放入数据库。之后，结合其他因子，考虑其相关性和对组合的贡献，最终形成实盘策略。不同的因子在排序打分时具有不同的权重，这些权重基于因子的历史表现和因子之间的相关性进行调整。\n\n2. **机器学习模型**：\n   - 管理人使用机器学习模型来预测不同子策略的表现，并据此调整策略权重。具体来说，机器学习用于分析多个策略在一段时间内的表现，以预测未来一周各策略的表现，并据此调整权重。\n   - 管理人还提到了使用机器学习来处理因子，尽管大部分因子目前是手工挖掘的，但有同事正在研究机器学习在因子挖掘方面的应用。\n\n3. **算法和用途**：\n   - **机器学习算法**：用于预测策略表现和可能的权重调整。具体算法未在文档中提及，但提到了使用第三方的机器学习服务。\n   - **T0增强**：采用第三方提供的T0增强服务，这可能涉及到高频信号转换成低频信号的处理，以及日频的调仓决策。\n\n4. **偏好**：\n   - 管理人目前主要使用线性模型进行因子的排序打分，但也在探索机器学习模型的应用，特别是在策略预测和权重调整方面。这表明管理人在建模方法上持开放态度，愿意结合传统线性模型和新兴的机器学习技术。\n\n总结来说，管理人的建模方法结合了线性模型和机器学习模型，以实现更精准的策略预测和权重调整。他们使用排序打分进行股票评估，同时利用机器学习算法来优化策略表现和调整策略权重。'

In [42]:
integrate_assistant = ConversableAgent(
        name="报告撰写员",
        system_message=f"你是一个报告撰写员，需要根据我发给你的字段，根据我的要求，详细的写出一份报告，并返回给我,以下是要求：不需要面面俱到，写出提供材料中包含的即可：{Prompt_dics.Strategy_requirements_dic['Simple Quant Strategy']}",
        llm_config = LLM_configs.Qwen_strong_llm_config
    )

In [13]:
print(Prompt_dics.Strategy_requirements_dic['Simple Quant Strategy'])

以下是量化策略的模板，以下内容都需要在段落中进行体现, 管理人的量化模型和因子上需要写详细一些：1. 管理⼈量化策略的整体思路，这部分主要说明管理人的量化策略的核心思路2. 管理⼈的因⼦组成百分比和因子的总数量, 以及管理人对对应因子的描述。3. 管理⼈的因⼦挖掘⽅法, 具体需要判断是人工手工挖掘，还是使用了机器学习，并详述他们用到的算法和方法论4. 管理⼈的因⼦处理⽅法, 具体需要判断其对因子的合成方法，处理方法等（如有），并详述他们用到的算法和方法论5. 管理⼈的量化模型，需要明确提示是线性还是机器学习模型或者是两者都有及其偏好。并详述他们的建模方法论，包括使用了何种模型，方法或算法进行了何种任务6. 管理⼈的因⼦⼊库条件, 即管理人的新因子入库标准和流程 7. 管理⼈的具体执⾏, 对股票进行了什么处理，有无使用算法下单等，最终持有了多少个股，指数内占比，行业偏离度，风格偏离度，年化双边换手率多少倍。8. 管理⼈的对冲⽅法（如有）， 使⽤什么进⾏对冲, 有⽆敞⼝
